**ВАЖНЫЙ КОММЕНТАРИЙ**

Обратите внимание:

* в датафрейме отсутсвует столбец stage_4_output_danger_gas;

* В ходе поспешного изменения, не успел убрать/изменить некоторые обоснования, которые теперь некорректны для данного кода.

* значения в DataTime были нормализированы. То есть приведены в диапазон от 0 до 1.

* Индесы были сброшены(ресетнуты), из-за чего, например, строка с иднексом 6 стала с индексом 5. Но есть список индексов(до сброса/ресета) **index_series**, который появляется в разделе
"Замена пропусков". Я так понимаю, что после всех моих строчек кода вам нужно будет изменить индесы датафрема на индексы в этом списке для корректной работы. После этого можно будет так же привести DateTime столбец к типу DateTime аналогичным способом.


Сейчас сижу в метро с плохим интернетом. Не могу проверить некоторые изменения, связанные с index_series, поэтому вам придётся принять данную ношу.



# Подготовка данных для предсказания и замены чисел вместо пропусков

## Импорт библиотек

In [ ]:
!pip install miceforest

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import miceforest as mf
from sklearn.base import clone
from sklearn.preprocessing import MinMaxScaler
from itertools import combinations
from scipy.stats import spearmanr, mannwhitneyu
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, r2_score


## Важные блоки кода. Функции/Классы

### **функции**

In [ ]:
def nan_to_median(series: pd.Series):
  median_val = series.median()
  return series.fillna(median_val)

def iqr_filter(df: pd.DataFrame, column: str, lower_bound=True, upper_bound=True, fill_nan=False, multp=3):
  """
  гибкая функция для удаления выбросов с помощью настраиваемого интерквартильного размаха

  Аргументы
      df: DataFrame содержащий данные, которые нужно отфильтровать.

      column: Название столбца в DataFrame df, в которомом будет производиться фильтрация.
      Функция будет рассчитывать IQR именно для этого столбца.

      lower_bound:  Булевый флаг, определяющий, следует ли отфильтровывать значения,
      лежащие ниже нижней границы, рассчитанной на основе IQR. То есть, если
      lower_bound = False, то все выбросы(если они есть) будут игнорироваться.

      upper_bound: Булевый флаг, определяющий, следует ли отфильтровывать значения,
      лежащие выше верхней границы, рассчитанной на основе IQR.

      multp: Множитель, используемый для расчета границ фильтрации.
  """
  df_copy = df.copy()

  if fill_nan:
    df_copy[column] = nan_to_median(df_copy[column])

  q1, q3 = np.percentile(df_copy[column], [25, 75])
  iqr = (q3 - q1) * multp
  low_bound = q1 - iqr
  up_bound = q3 + iqr

  median = 0 # медиана
  outlines = 0 # индексы, значения строк столбца которых необходимо заменить медианой

  # в этих ситуациях мы ищем медиану чисел, которые не считаем за выбросы (1)
  # то есть мы считаем за выбросы up_bound или/и low_bound и не используем их диапазон значений
  # для поиска медианы.
  if lower_bound and upper_bound:
    outlines = df_copy[(df_copy[column] < low_bound) | (df_copy[column] > up_bound)].index
    median = df_copy[(df_copy[column] >= low_bound) & (df_copy[column] <= up_bound)][column].median() # та самая медиана (1)
  elif lower_bound:
    outlines = df_copy[df_copy[column] < low_bound].index
    median = df_copy[df_copy[column] >= low_bound][column].median() # та самая медиана (1)
  elif upper_bound:
    outlines = df_copy[df_copy[column] > up_bound].index
    median = df_copy[df_copy[column] <= up_bound][column].median() # та самая медиана (1)

  df.loc[outlines, column] = median


####################################################################################################


def find_dependencies(correlation_matrix, threshold=0.5):
    """
    Выводит пары зависимых признаков на основе матрицы корреляции.

    Аргументы:
        correlation_matrix: Pandas DataFrame с матрицей корреляции.
        threshold: Порог корреляции для определения зависимости.
    """

    dependent_features = {} # словарь для хранения выявленных зависимостей.

    for feature in correlation_matrix.columns:  # цикл переберает все столбцы в матрице
        correlations = correlation_matrix[feature] # для текущей feature эта строка извлекает
                                                   # значения корреляции этой функции со всеми
                                                   # остальными функциями в виде pd.Series .
        dependencies = correlations[correlations.abs() >= threshold].index.tolist() # основная логика
        #correlations.abs() >= threshold: Это создаёт логическую маску, которая выбирает только
        #значения корреляции, которые по модулю абсолютного значения больше или равны threshold.
        #.index.tolist(): .index извлекает индексы (имена признаков), соответствующие
        #выбранным значениям корреляции, и .tolist() преобразует эти индексы в список.


        if dependencies:  # если есть зависимости
          dependencies.remove(feature) # удаляем зависимость с самим собой
          dependent_features[feature] = dependencies # выявленные зависимости сохраняются
                                                     # в словаре dependent_features.
                                                     # Текущий признак становится ключом,
                                                     # а список его зависимостей — значением.
    return dependent_features # пары зависимых признаков на основе матрицы корреляции.


####################################################################################################


def visualize_correlation_matrix(correlation_matrix, title="The best graphic"): # красивый график ВСЕХ корреляций(колличественных)
    """
    Рисует график корреляций всех колличественных признаков(и не только, если надо).

    Аругменты:
        correlation_matrix:  Фрейм данных Pandas, представляющий корреляционную матрицу.
        threshold: Название графика.
    """
    for col in correlation_matrix.columns: # цикл повторяется по каждому столбцу корреляционной матрицы.
        correlation_matrix[col] = pd.to_numeric(correlation_matrix[col], errors='coerce') # спасает от вылета программы.

    #correlation_matrix_np = correlation_matrix.values # преобразует фрейм данных Pandas в массив NumPy

    plt.figure(figsize=(10, 8))
    plt.imshow(correlation_matrix, cmap='coolwarm', interpolation='nearest') # создание тепловой карты корреляций

    plt.colorbar(label='Correlation')

    plt.xticks(range(len(correlation_matrix.columns)), correlation_matrix.columns, rotation=90)
    plt.yticks(range(len(correlation_matrix.index)), correlation_matrix.index)

    plt.title(title)
    plt.show()


####################################################################################################



def super_train_test_split(df: pd.DataFrame, y: pd.Series):
  '''
  Делит данные на две выборки: 1. строки, значения необходимого нам столбца не имеют пропусков.
                               2. строки, значения необходимого нам столбца имеют пропуски.
  Каждый из этих пунктов так же делиться на две выборки: а) необходимый столбец.
                                                         б) остальные факторы.

  Аргументы:
    df: Pandas DataFrame, состоящий из факторов, инмеющих зависимость с признаком,
        в котором необходимо заполнить пропуски.

    y: Pandas Series, признак, пропуски которого необходимо заполнить.

  небольшой комментарий:
  У нас есть проблема - для заполенния пропусков с помощью какой-либо модели, необходимо,
  чтобы ВСЕ значения в других признаках были заполнены(не было пропусков).
  В противном случае модель ругается, что есть NaNы. Данный цикл устраняет данную проблему,
  временно заполняя пропуски в столбцах на медиану всех значений признака
  (кроме столбца, задача для которого изначально была заполнить пропуски с помощью модели).
  Дальше смотрите по комментариям
  '''
  X = df.copy()

  for col in X.columns: # временно заменяем пропуски в зависимых факторах на медиану
    if X[col].isnull().any() == True:
      median_value = X[col].median()
      X[col] = X[col].fillna(median_value)


  y_train = y[y.isnull() == False] # отбираем для тренировки те строки, в которых присутсвуют данные
  y_temp = y[y.isnull()] # просто мусор. Полезный

  idxs = y_temp.index # берём иднексы мусора(индексы,
                      # в строках которых есть пропуски, которые необходимо заполнить)
  X_train = X.drop(idxs) # делаем обучающую выборку из строк, в которых нет пропусков

  idxs = y_train.index # берём иднексы c изначально заполенными значениями
  X_test = X.drop(idxs) # отбрасываем строки с заполненными значениями в нужном нам столбце.
                        # Получается выборка с данными, на основе которых будут
                        # предсказываться пропущенные значения

  return X_train, X_test, y_train, y_temp


####################################################################################################


def split_for_grade(df: pd.DataFrame, target_column: pd.Series): # без комментариев, это вам не пригодится
  X = df.copy()

  y = target_column
  y1 = y[y.isnull() == False] # отбираем для тренировки те строки, в которых присутсвуют данные
  y_temp = y[y.isnull()] # просто мусор. Полезный

  idxs = y_temp.index # берём иднексы мусора(индексы,
                      # в строках которых есть пропуски, которые необходимо заполнить)
  X = X.drop(idxs) # делаем обучающую выборку из строк, в которых нет пропусков
  for col in X.columns:
    if X[col].isnull().any() == True:
      median_value = X[col].median()
      X[col] = X[col].fillna(median_value)

  X_train, X_test, y_train, y_test = train_test_split(
    X, y1, test_size=len(y_temp) / len(y1), random_state=42)
  return X_train.values, X_test.values, y_train.values, y_test.values

### ***Классы***

In [ ]:
class SBS():
    """
    Класс для последовательного обратного отбора признаков (Sequential Backward Selection).

    Алгоритм отбирает подмножество наиболее важных признаков,
    оптимизируя модель по метрикам качества (R-квадрат и MSE).

    Аргументы:
        estimator: Модель машинного обучения, которую нужно оптимизировать.
                   Должна поддерживать методы fit и predict.
        k_features: Целевое количество признаков для отбора.
        test_size: Доля данных для тестирования (кросс-валидация).
        random_state: Случайное зерно для воспроизводимости результатов.

    """
    def __init__(self, estimator, k_features,
                test_size=0.25, random_state=42):
        self.estimator = clone(estimator) # Создаём копию модели, чтобы не менять исходную
        self.k_features = k_features
        self.test_size = test_size
        self.random_state = random_state

    def fit(self, X, y, own_split=False):
        """
        Обучает модель SBS и отбирает лучшие признаки.

        Args:
            X: Матрица признаков.
            y: Вектор целевой переменной.
            own_split: Если True, использует пользовательскую функцию split_for_grade для разделения данных.

        Returns:
            Возвращает себя (self) для цепочки вызовов.
        """

        # Разделение данных на обучающую и тестовую выборки
        X_train, X_test, y_train, y_test = split_for_grade(X, y)

        dim = X_train.shape[1]
        self.indices_ = tuple(range(dim))  # Индексы всех признаков
        self.subsets_ = [self.indices_] # Список всех подмножеств признаков

        # Вычисляем R-SQUARED и MSE
        score = self._calc_score(X_train, y_train,
                                    X_test, y_test, self.indices_)
        self.scores_ = [score]

        while dim > self.k_features:
            scores = []
            subsets = []

            # Перебор всех возможных подмножеств с одним удаленным признаком
            for p in combinations(self.indices_, r=dim - 1):
                score = self._calc_score(X_train, y_train,
                                            X_test, y_test, p)
                scores.append(score)
                subsets.append(p)

            # находим подмножества с лучшими значениями метрик
            best = np.argmax([i[0] for i in scores]) #  Выбираем подмножество с наибольшим r2_score,
                                                     # т.к. данная метрика в приоритете. Так же отбор
                                                     # лучшей комбинации будет происходит вне класса
            self.indices_ = subsets[best]
            self.subsets_.append(self.indices_)
            dim -= 1

            self.scores_.append(scores[best])
        self.k_score_ = self.scores_[-1]

        return self

    def transform(self, X):
        """
        Возвращает матрицу признаков с отобранными признаками.

        Аргументы:
            X: Матрица признаков.
        """
        return X[:, self.indices_]

    def _calc_score(self, X_train, y_train, X_test, y_test, indices):
        """
        Вычисляет метрики R-SQUARED и MSE для заданного подмножества признаков.

        Аргументы:
            X_train, y_train, X_test, y_test, indices: Данные для обучения и оценки модели.
        """
        self.estimator.fit(X_train[:, indices], y_train)
        y_pred = self.estimator.predict(X_test[:, indices])

        score = [r2_score(y_test, y_pred), mean_squared_error(y_test, y_pred)]
        return score


## Инициализация моделей и главного дата фрейма

инициализация регрессионных моделей и т.п.

In [ ]:
random_forest_model = RandomForestRegressor(random_state=42)
linear_regression = LinearRegression()
knn_model = KNeighborsRegressor(n_neighbors=3)
scaler = MinMaxScaler()

In [ ]:
df_environmental_data = pd.read_csv("analysing_environmental_issues.csv", sep=',') # главный датафрейм

In [ ]:
#df_environmental_data

## предобработка перед оценкой моделей и предсказыванием значений

Просто копия главного датафрейма для экспериментов

In [ ]:
df = df_environmental_data.copy()

Регрессионным моделям не нравится столбец DateTime, поэтому я его пока что просто удалил, но можно поставить в качестве индекса. DateTime сам по себе не очень полезен(на этапе предобработки).   

In [ ]:
df['DateTime'] = pd.to_datetime(df['DateTime'], errors='coerce')
time_diffs = df['DateTime'].diff().dt.total_seconds()
time_diffs = time_diffs.fillna(0)

# нормализуем даты из столбца DateTime
normalized_diffs = scaler.fit_transform(time_diffs.values.reshape(-1, 1)).flatten()

# вычисляет кумулятивную сумму элементов
normalized_times = np.cumsum(normalized_diffs)

# подставляем нормализованные значение
df['DateTime'] = normalized_times

df.pop("stage_4_output_danger_gas") # значений мало, признак на данном этапе бесполезный.

df = df.drop_duplicates(subset=df.columns[1:], keep=False) # удаляем все дупликаты.
# Очень важно, что дубликаты будут найдены, если игнорировать столбец "DateTime".
df

In [ ]:
# удаляем супер выбросы/критические ошибки, так как такие значения
# могут непропорционально влиять на оценку параметров модели и
# вносить нестабильность в обучение модели. Для этого используем
# тройной межквартильный размах
for col in df.columns[1:-1]:
  iqr_filter(df, col, fill_nan=True, multp=3)

#df.describe()

Было протестированно множество разных моделей. В конченом счёте лучшими моделями оказались KNN и Random Trees Forest. В ходе размышлений было принято использовать KNN для тех признаков, если метрики R-SQUAED и MSE показали хорошие значения в тестовых выборках. Для отбора признаков в зависимости от которых будут предсказываться значения на месте пропусков в отклике.

## **SBS  для выбора факторов , на основе которых буддет обучаться модель и предсказываться значения**

**KNeighborsRegressor**

In [ ]:
columns_dict_NaN = {}

In [ ]:
sbs = SBS(knn_model, k_features=1)

for column in df.columns:
    if df[column].isnull().any() == True:
        y = df[column]
        X = df.copy()
        X.pop(column)

        new_names = [i for i in range(len(df.columns))]
        X = X.rename(columns=dict(zip(X, new_names)))
        sbs.fit(X, y, own_split=True)

        k_feat = []
        r2 = []
        mse = []


        for scores in sbs.scores_:
            if scores[1] < 10 and scores[0] > 0.7:
                k_feat.append(len(sbs.subsets_[sbs.scores_.index(scores)]))
                r2.append(scores[0])
                mse.append(scores[1])

        if len(mse) > 0:

            r2_threshold = 0.7
            mse_threshold = 10

            # Normalize MSE values
            mse_values = np.array([item[1] for item in sbs.scores_])
            scaler = MinMaxScaler()
            mse_values_normalized = scaler.fit_transform(mse_values.reshape(-1, 1)).flatten()

            best_r2 = -1
            best_mse = float('inf')
            best_pair = None
            lk = -1
            for i, (r2_sc, mse_sc) in enumerate(sbs.scores_):
                normalized_mse = mse_values_normalized[i]
                rounded_mse = round(normalized_mse, 3)

                if r2_sc > r2_threshold and mse_sc <= mse_threshold:
                    if r2_sc > best_r2 and rounded_mse < best_mse:
                        best_r2 = r2_sc
                        best_mse = rounded_mse
                        best_pair = [r2_sc, mse_sc]
                        lk = list(sbs.subsets_[sbs.scores_.index([r2_sc, mse_sc])])

            print(f"Best count: {len(lk)}")
            print(f"Best pair: R2 = {best_pair[0]:.4f}, MSE = {best_pair[1]:.4f}, Normalized MSE = {best_mse:.4f}")

            columns_dict_NaN[column] = list(df.columns[0:][lk])
            #dk[column] = X.columns[1:, list(sbs.subsets_[10])]

            plt.figure(figsize=(10, 6))  # Adjust figure size as needed

            # Plot R-squared
            plt.plot(k_feat, r2, marker='o', linestyle='-', label='R-squared')

            # Plot MSE
            plt.plot(k_feat, mse, marker='x', linestyle='-', label='MSE')

            plt.xlabel("Number of Features")
            plt.ylabel("Metric Value")
            plt.title(column)
            plt.legend()
            plt.grid(True)
            plt.show()

print(columns_dict_NaN)

In [ ]:
sbs = SBS(random_forest_model, k_features=1)

for column in df.columns:
    if df[column].isnull().any() == True:
        y = df[column]
        X = df.copy()
        X.pop(column)

        new_names = [i for i in range(len(df.columns))]
        X = X.rename(columns=dict(zip(X, new_names)))
        sbs.fit(X, y, own_split=True)

        k_feat = []
        r2 = []
        mse = []


        for scores in sbs.scores_:
            if scores[1] < 10 and scores[0] > 0.7:
                k_feat.append(len(sbs.subsets_[sbs.scores_.index(scores)]))
                r2.append(scores[0])
                mse.append(scores[1])

        if len(mse) > 0:

            r2_threshold = 0.7
            mse_threshold = 10

            # Normalize MSE values
            mse_values = np.array([item[1] for item in sbs.scores_])
            scaler = MinMaxScaler()
            mse_values_normalized = scaler.fit_transform(mse_values.reshape(-1, 1)).flatten()

            best_r2 = -1
            best_mse = float('inf')
            best_pair = None
            lk = -1
            for i, (r2_sc, mse_sc) in enumerate(sbs.scores_):
                normalized_mse = mse_values_normalized[i]
                rounded_mse = round(normalized_mse, 3)

                if r2_sc > r2_threshold and mse_sc <= mse_threshold:
                    if r2_sc > best_r2 and rounded_mse < best_mse:
                        best_r2 = r2_sc
                        best_mse = rounded_mse
                        best_pair = [r2_sc, mse_sc]
                        lk = list(sbs.subsets_[sbs.scores_.index([r2_sc, mse_sc])])

            print(f"Best count: {len(lk)}")
            print(f"Best pair: R2 = {best_pair[0]:.4f}, MSE = {best_pair[1]:.4f}, Normalized MSE = {best_mse:.4f}")

            columns_dict_NaN[column] = list(df.columns[0:][lk])


print(columns_dict_NaN)

In [ ]:
print(len(columns_dict_NaN.keys()))

# Замена пропусков

In [ ]:
df

главный гость нашей программы. Весь код ранее был нужен для корректной работы этих строчек кода.
Здесь мы заменяем NaNы на прогнозируемые значения

In [ ]:
for column in columns_dict_NaN.keys(): # для каждого столбца в котором остались пропущенные значения
    print(column)
    X = df.loc[:, columns_dict_NaN[column]].copy() # все зависимые признаки с столбцом column ()
    y = df[column] # наш столбец.

    X_train, X_test, y_train, y_temp = super_train_test_split(X, y) # Делим наши данные на
                                                                    # на обучающую и тестовую выборку
    knn_model.fit(X_train, y_train) # обучаем модель knn(для каждого столбца)

    y_pred = knn_model.predict(X_test) # предсказываем пропущенные значения

    df.loc[X_test.index, column] = y_pred # вставляем предсказания на места пропущенных значений

In [ ]:
index_list = df.index.tolist()
index_series = pd.Series(index_list)
df = df.reset_index(drop=True)

In [ ]:
kernel = mf.ImputationKernel(
    data=df, #Useful for convergence analysis.
    random_state=42  #For reproducibility
)

# Perform MICE imputation. Experiment with 'iterations'.  Too few might not converge,
# too many might be computationally expensive with diminishing returns.
kernel.mice(iterations=10) # Number of iterations

# Get the completed datasets
df = kernel.complete_data()

df

In [ ]:
df.describe() # просто смотрю на изменение данных

In [ ]:
df

# Как интегрировать это чюдо в свой код

Насколько я понимаю, вам достаточно скопировать всё эти махинации в самое начало вашего блокнота и после последней строчки моего кода написать
"название вашей переменной с датафреймом" = df


In [ ]:
#my_data = df

А как буд-то мы можем себя возомнить гениями и в итоговом проекте сделать отдельный скрипт с этим кодом, который будет записывать в файлик csv Датафрейм без пропусков(или использовать данный скрипт как библиотеку и написать здесь функцию, которая будет возвращать обработанный дата фрейм), чтобы не загромождать один файл кодом.